In [58]:
import pandas as pd
import numpy as np

In [59]:
site_df = pd.read_csv('../data/DeDe_48stations/DeDe_48sites_metadata.csv')

In [62]:
# merge CAMS, ERA5 and DeDe data (15 min)

for i,row in site_df.iterrows():

    # Load CAMS data
    cams_df = pd.read_csv(f'../data/CAMS/cams_time_series_15min/cams_solar_rad_alti_8_2023_site{row['Site_id']:02d}_15min.csv',parse_dates=['Datetime'])
    cams_df.set_index('Datetime', inplace=True)
    # cams_df = cams_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    cams_df = cams_df[cams_df.index.year == 2023]
    cams_df.reset_index(inplace=True)

    # Load ERA5 data
    era5_df = pd.read_csv(f'../data/ERA5/radiation/all_site_15min/era5_15min_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_48stations/merge_15min_all48sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes on 'Datetime'
    merged_df = pd.merge(cams_df, era5_df, on='Datetime', suffixes=('_cams', '_era5'),how='outer')
    merged_df = pd.merge(merged_df, dede_df, on='Datetime', suffixes=('', '_dede'),how='outer')



    merged_df.drop(columns=[ 'TOA', 'Clear sky GHI', 'Clear sky BHI', 'Clear sky DHI',
        'Clear sky BNI', 'BHI', 'DHI', 'BNI', 'Reliability', 'Site_id', 'rawI',
        'CI_CM', 'CI_CM_interpolated','CI_RGB', 'CI_RGB_interpolated', 'Temperature', 'Relative Humidity',
        'Pressure', 'Wind speed', 'Wind direction', 'Rainfall', 'Snowfall',
        'Snow depth', 'Iclr'],
        inplace=True, errors='ignore')

    # Save merged data
    merged_df.to_csv(f'../merged_data/solar_radiation_15min/solar_radiation_15min_site{row['Site_id']:02d}_2023.csv', index=False)

In [63]:
# merge CAMS, ERA5 and DeDe data (1 hr)
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='1h')
for i,row in site_df.iterrows():

    # Load CAMS data
    cams_df = pd.read_csv(f'../data/CAMS/cams_time_series_1hr/cams_solar_rad_alti_8_2023_site{row['Site_id']:02d}_1hour.csv',parse_dates=['Datetime'])
    cams_df.set_index('Datetime', inplace=True)
    # cams_df = cams_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    cams_df = cams_df[cams_df.index.year == 2023]
    cams_df.reset_index(inplace=True)

    # Load ERA5 data
    era5_df = pd.read_csv(f'../data/ERA5/radiation/all_site_1hr/era5_1hr_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_48stations/merge_15min_all48sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])
    dede_df = dede_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='inner')

    # Merge dataframes on 'Datetime'
    merged_df = pd.merge(cams_df, era5_df, on='Datetime', suffixes=('_cams', '_era5'),how='outer')
    merged_df = pd.merge(merged_df, dede_df, on='Datetime', suffixes=('', '_dede'),how='outer')

    merged_df.drop(columns=[ 'TOA', 'Clear sky GHI', 'Clear sky BHI', 'Clear sky DHI',
        'Clear sky BNI', 'BHI', 'DHI', 'BNI', 'Reliability', 'Site_id', 'rawI',
        'CI_CM', 'CI_CM_interpolated','CI_RGB', 'CI_RGB_interpolated', 'Temperature', 'Relative Humidity',
        'Pressure', 'Wind speed', 'Wind direction', 'Rainfall', 'Snowfall',
        'Snow depth', 'Iclr'],
        inplace=True, errors='ignore')

    # Save merged data
    merged_df.to_csv(f'../merged_data/solar_radiation_1hr/solar_radiation_1hr_site{row['Site_id']:02d}_2023.csv', index=False)

In [ ]:
# merge CAMS_forcast and DeDe data (1 hr)
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='1h')
for i,row in site_df.iterrows():

    # Load CAMS_forcast data
    cams_df = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_ssrd1hr_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_48stations/merge_15min_all48sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes on 'Datetime'
    cams_df = cams_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='outer')
    merged_df = pd.merge(cams_df, dede_df, on='Datetime', suffixes=('_cams', '_era5'),how='inner')
    merged_df.drop(columns=['Site_id','CI_CM',
        'CI_CM_interpolated', 'CI_RGB', 'CI_RGB_interpolated', 'Temperature',
        'Relative Humidity', 'Pressure', 'Wind speed', 'Wind direction',
        'Rainfall', 'Snowfall', 'Snow depth', 'Short-wave irradiation','rawI'], inplace=True)

    # Save merged data
    merged_df.to_csv(f'../merged_data/cams_forecast_1hr/cams_forecast_1hr_site{row['Site_id']:02d}_2023.csv', index=False)

In [33]:
# merge CAMS_forcast and DeDe data (15 min)
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='15min')
for i,row in site_df.iterrows():

    # Load CAMS_forcast data
    cams_df = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_ssrd15min_2023_site{row['Site_id']:02d}.csv', parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_48stations/merge_15min_all48sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes on 'Datetime'
    cams_df = cams_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='outer')
    merged_df = pd.merge(cams_df, dede_df, on='Datetime', suffixes=('_cams', '_era5'),how='inner')
    merged_df.drop(columns=['Site_id','CI_CM',
        'CI_CM_interpolated', 'CI_RGB', 'CI_RGB_interpolated', 'Temperature',
        'Relative Humidity', 'Pressure', 'Wind speed', 'Wind direction',
        'Rainfall', 'Snowfall', 'Snow depth', 'Short-wave irradiation','rawI'], inplace=True)
    # Save merged data
    merged_df.to_csv(f'../merged_data/cams_forecast_15min/cams_forecast_15min_site{row['Site_id']:02d}_2023.csv', index=False)

Processing site: กรมพัฒนาพลังงานทดแทน และ อนุรักษ์พลังงาน กรุงเทพมหานคร
Processing site: สถานีอุตุนิยมวิทยานครราชสีมา
Processing site: ศูนย์อุตุนิยมวิทยาภาคตะวันออกเฉียงเหนือตอนบน จ.ขอนแก่น
Processing site: สถานีอุตุนิยมวิทยาร้อยเอ็ด
Processing site: สถานีอุตุนิยมวิทยาอุบลราชธานี
Processing site: สถานีอุตุนิยมวิทยานครพนม
Processing site: ส่วนอุทกวิทยาที่ 1 จ.หนองคาย
Processing site: สถานีอุตุนิยมวิทยาเลย
Processing site: ศูนย์บริการวิชาการที่ 7 จ.เชียงใหม่
Processing site: ศูนย์บริการวิชาการที่ 8 จ.พิษณุโลก
Processing site: สถานีอุตุนิยมวิทยาเพชรบูรณ์
Processing site: สถานีอุตุนิยมวิทยาหนองพลับ จ.ประจวบคีรีขันธ์
Processing site: สถานีอุตุนิยมวิทยาระนอง
Processing site: ศูนย์อุตุนิยมวิทยาภาคใต้ฝั่งตะวันตก  จ.ภูเก็ต
Processing site: ศูนย์บริการวิชาการที่ 10 จ.สงขลา 
Processing site: สถานีอากาศเกษตรสุรินทร์
Processing site: สถานีอุตุนิยมวิทยาน่าน
Processing site: สถานีอุตุนิยมวิทยาทองผาภูมิ จ.กาญจนบุรี
Processing site: สถานีอุตุนิยมวิทยากาญจนบุรี
Processing site: สถานีอุตุนิยมวิทยาแพร่
Pr

In [ ]:
# merge cloud เขียนเองนะ